In [ ]:
import dspy
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
lm = dspy.LM("openrouter/deepseek/deepseek-chat-v3-0324:free",api_key=os.getenv("OPEN_ROUTER_API_KEY_2"))
dspy.configure(lm=lm)

In [ ]:
class RandomTaskCreationSignature(dspy.Signature):
    context:str = dspy.InputField()
    created_random_tasks:list[str] = dspy.OutputField()

context = """
I am going to test the interactions and the workflow of a multi agent system(MAS) and need tasks for the agents to work on.
These tasks should be about software development, such as writing code, reviewing code, debugging, or designing software architecture.
The tasks should be diverse and make sure that all the roles in the MAS can be utilized.(e.g. developer, architect, project manager, project manager etc.)
You can achieve this by this by creating a list of steps such as create system specification, write code, review code, test code, deploy code, etc.
Make sure that the relevant result is returned as output. Ensure that the tasks contain all the relevant context and information that the agents need to complete the task.
Do not create tasks that rely on external information or resources that are not available in the current context.

Exampple task:
Give me the system design properties and the relevant .py file for a calculator program that receives input from the console. 
Ask help from the software architect agent by getting its details. 
Give me the summary of the resulting code and the system specifications, and the list of other agents that you interacted with.

"""

random_tasks_prediction = dspy.Predict(RandomTaskCreationSignature)

tasks = random_tasks_prediction(context=context).created_random_tasks
tasks

In [ ]:
import requests

# send the tasks to the spring boot server
server_url = os.getenv("SPRING_SERVER_URL")

if server_url:

    for i, task in enumerate(tasks):
        print(f"Task {i+1}: {task}")

        response = requests.post(
            f"{server_url}/startworkflow",
            json={
                "sender": "task_generator",
                "message": task
                }
        )
        if response.status_code == 200:
            print("Task completed successfully.")
            print("Response: ", response.text)

        else:
            print(f"Failed to send tasks. Status code: {response.status_code}")